In [1]:
import os
import sys
import collections
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import time
from pymongo import MongoClient
import urllib
import multiprocess
import pickle
import random

# BERT files
os.listdir("../bert-master")
sys.path.insert(0, '../bert-master')

from run_classifier import *
import modeling
import optimization
import tokenization

import preprocessor as p


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
p.set_options(p.OPT.URL, p.OPT.EMOJI)

ModuleNotFoundError: No module named 'pandas'

In [2]:
# Set up data directories
data_dir = './data'
model_dir = './model'
output_dir = './output'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
if not os.path.exists(model_dir):
    os.makedirs(data_dir)

In [6]:
username = 'christian'
password = 'Dec211996'

client = MongoClient('mongodb://' + urllib.parse.quote_plus(username) + ':' + urllib.parse.quote_plus(password) + '@198.211.115.252')

In [34]:
def worker(consp_tuple): 
    
    ## this is temp
    from pymongo import MongoClient
    import urllib
    import pickle
    username = 'christian'
    password = 'Dec211996'

    client = MongoClient('mongodb://' + urllib.parse.quote_plus(username) + ':' + urllib.parse.quote_plus(password) + '@198.211.115.252')
    
    data_dir = './data'
    ## end temp
    
    data = []
    
    conspiracy, hashtag = consp_tuple
    cursor = client[conspiracy][hashtag].find({})
    for i, document in enumerate(cursor):
        try:
            inputs = {'text' : p.clean(document['text'])}
            inputs.update({'tweetId' : document['tweetId']})
            data.append(inputs)
        except Exception as e:
            pass


        if (i > 100):
            break

    with open(data_dir + '/' + conspiracy + '-' + hashtag + '.p', 'wb') as f:
        pickle.dump(data, f)

    return consp_tuple

def get_consp_tuples():
    username = 'christian'
    password = 'Dec211996'

    client = MongoClient('mongodb://' + urllib.parse.quote_plus(username) + ':' + urllib.parse.quote_plus(password) + '@198.211.115.252')
    
    ignore_mask = ['test_db', 'admin', 'local', 'config', 'TwitterJobs']
    conspiracies = list(set(client.list_database_names()) - set(ignore_mask))

    consp_tuples = []
    for conspiracy in conspiracies:
        for hashtag in client[conspiracy].list_collection_names():
            consp_tuples.append((conspiracy, hashtag))

    return consp_tuples

def preprocess():
    consp_tuples = get_consp_tuples()
    random.shuffle(consp_tuples)
    p = multiprocess.Pool(multiprocess.cpu_count())
#     consp_tuples = [(consp_tuple, data_dir) for consp_tuple in consp_tuples]
#     results = p.map(worker, consp_tuples)

    for i in range(10):
        worker(consp_tuples[i])

preprocess()

In [3]:
def generate_labels():
    conspiracies = set()
    
    for filename in os.listdir(data_dir):
        if '.p' in filename:
            conspiracies.add(filename.split('-')[0])
    return {consp:i for i, consp in enumerate(conspiracies)}

In [6]:
data = []
dummytext = '*'

labels = generate_labels()

for filename in os.listdir(data_dir):
    if '.p' in filename:
        with open(data_dir + '/' + filename, 'rb') as f:
            consp = filename.split('-')[0]
            x = pickle.load(f)
            for row in x:
                data.append({'dummy_1': row['tweetId'], 'target': labels[consp], 'dummy_2' : dummytext, 'text' : row['text']})
df = pd.DataFrame(data)
data = None
print(df)

                   dummy_1  target dummy_2  \
0               7311643605      11       *   
1               7285727966      11       *   
2               7507472369      11       *   
3               7507355731      11       *   
4               7507321301      11       *   
...                    ...     ...     ...   
235617  318870412580761600       7       *   
235618  318828537354260480       7       *   
235619  318604933932916736       7       *   
235620  315200468123009024       7       *   
235621  314913822756442112       7       *   

                                                     text  
0       Alex Jones: Tribute to George #Washington #fou...  
1       Going back in time to November 09, 2005.In thi...  
2       Ron Paul on the Alex Jones Radio Show #tlot #t...  
3       A Tax Trap! #tlot #tcot #irs #tax #tyranny #li...  
4       Crazy theories? #tlot #tcot #tporg #ronpaul #a...  
...                                                   ...  
235617  White Student Union

In [7]:
# force train into cola format, test is fine as it is
# train = train.sample(frac=0.01)
# test = train.sample(frac=0.01)

test_split = 0.2

permuted_indices = list(np.random.permutation(len(df)))
test_indices = random.sample(permuted_indices, int(len(permuted_indices) * test_split))
train_indices = list(set(permuted_indices) - set(test_indices))

train = df.iloc[train_indices]
test = df.iloc[test_indices]

train.to_csv(data_dir + '/train.tsv', sep='\t', index=False, header=False)
test.to_csv(data_dir + '/test.tsv', sep='\t', index=False, header=True)

In [8]:
class MultiClassColaProcessor(ColaProcessor):
    def get_labels(self):
        """See base class."""
        return list([str(x) for x in labels.values()])

In [9]:
task_name = 'cola'
bert_config_file = model_dir + '/bert_config.json'
vocab_file = model_dir + '/vocab.txt'
init_checkpoint = model_dir + '/bert_model.ckpt'
do_lower_case = True
max_seq_length = 72
do_train = True
do_eval = False
do_predict = False
train_batch_size = 32
eval_batch_size = 32
predict_batch_size = 32
learning_rate = 2e-5 
num_train_epochs = 1.0
warmup_proportion = 0.1
use_tpu = False
master = None
save_checkpoints_steps = 99999999 # <----- don't want to save any checkpoints
iterations_per_loop = 1000
num_tpu_cores = 8
tpu_cluster_resolver = None

In [11]:
start = time.time()
print("--------------------------------------------------------")
print("Starting training ...")
print("--------------------------------------------------------")

bert_config = modeling.BertConfig.from_json_file(bert_config_file)

processor = MultiClassColaProcessor()
label_list = processor.get_labels()

tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tpu_cluster_resolver = None
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
  cluster=tpu_cluster_resolver,
  master=master,
  model_dir=output_dir,
  save_checkpoints_steps=save_checkpoints_steps,
  tpu_config=tf.contrib.tpu.TPUConfig(
      iterations_per_loop=iterations_per_loop,
      num_shards=num_tpu_cores,
      per_host_input_for_training=is_per_host))

train_examples = processor.get_train_examples(data_dir)
num_train_steps = int(len(train_examples) / train_batch_size * num_train_epochs)
num_warmup_steps = int(num_train_steps * warmup_proportion)

model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=len(label_list),
      init_checkpoint=init_checkpoint,
      learning_rate=learning_rate,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=use_tpu,
      use_one_hot_embeddings=use_tpu)

estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_tpu,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=train_batch_size)
      
      
train_file = os.path.join(output_dir, "train.tf_record")

file_based_convert_examples_to_features(
    train_examples, label_list, max_seq_length, tokenizer, train_file)

tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", train_batch_size)
tf.logging.info("  Num steps = %d", num_train_steps)

train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=max_seq_length,
    is_training=True,
    drop_remainder=True)
    
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

end = time.time()
print("--------------------------------------------------------")
print("Training complete in ", end - start, " seconds")
print("--------------------------------------------------------")

--------------------------------------------------------
Starting training ...
--------------------------------------------------------
INFO:tensorflow:Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 99999999, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025C3852BD48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_

INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/key/bias:0, shape = (128,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/value/kernel:0, shape = (128, 128), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/value/bias:0, shape = (128,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/dense/kernel:0, shape = (128, 128), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/dense/bias:0, shape = (128,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/LayerNorm/beta:0, shape = (128,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/LayerNorm/gamma:0, shape = (128,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/intermediate/dense/kernel:0, shape = (128, 512), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/intermediate/dense/bias:0, shape = (512,), *INIT_FROM_CKPT*

INFO:tensorflow:global_step/sec: 7.75442
INFO:tensorflow:examples/sec: 248.141
INFO:tensorflow:global_step/sec: 8.40601
INFO:tensorflow:examples/sec: 268.992
INFO:tensorflow:global_step/sec: 8.15965
INFO:tensorflow:examples/sec: 261.109
INFO:tensorflow:global_step/sec: 8.19932
INFO:tensorflow:examples/sec: 262.378
INFO:tensorflow:global_step/sec: 8.33601
INFO:tensorflow:examples/sec: 266.752
INFO:tensorflow:global_step/sec: 8.06712
INFO:tensorflow:examples/sec: 258.148
INFO:tensorflow:global_step/sec: 8.54768
INFO:tensorflow:examples/sec: 273.526
INFO:tensorflow:global_step/sec: 8.40798
INFO:tensorflow:examples/sec: 269.055
INFO:tensorflow:global_step/sec: 8.19937
INFO:tensorflow:examples/sec: 262.38
INFO:tensorflow:global_step/sec: 8.33597
INFO:tensorflow:examples/sec: 266.751
INFO:tensorflow:global_step/sec: 7.93902
INFO:tensorflow:examples/sec: 254.049
INFO:tensorflow:global_step/sec: 8.17657
INFO:tensorflow:examples/sec: 261.65
INFO:tensorflow:global_step/sec: 8.26713
INFO:tensorfl

INFO:tensorflow:examples/sec: 252.048
INFO:tensorflow:global_step/sec: 7.98096
INFO:tensorflow:examples/sec: 255.391
INFO:tensorflow:global_step/sec: 8.33601
INFO:tensorflow:examples/sec: 266.752
INFO:tensorflow:global_step/sec: 8.26707
INFO:tensorflow:examples/sec: 264.546
INFO:tensorflow:global_step/sec: 8.33601
INFO:tensorflow:examples/sec: 266.752
INFO:tensorflow:global_step/sec: 8.33599
INFO:tensorflow:examples/sec: 266.752
INFO:tensorflow:global_step/sec: 8.47728
INFO:tensorflow:examples/sec: 271.273
INFO:tensorflow:global_step/sec: 7.3016
INFO:tensorflow:examples/sec: 233.651
INFO:tensorflow:global_step/sec: 7.52119
INFO:tensorflow:examples/sec: 240.678
INFO:tensorflow:global_step/sec: 8.09288
INFO:tensorflow:examples/sec: 258.972
INFO:tensorflow:global_step/sec: 8.06709
INFO:tensorflow:examples/sec: 258.147
INFO:tensorflow:global_step/sec: 8.06709
INFO:tensorflow:examples/sec: 258.147
INFO:tensorflow:global_step/sec: 7.35528
INFO:tensorflow:examples/sec: 235.369
INFO:tensorflow

INFO:tensorflow:examples/sec: 237.113
INFO:tensorflow:global_step/sec: 7.43696
INFO:tensorflow:examples/sec: 237.983
INFO:tensorflow:global_step/sec: 7.87648
INFO:tensorflow:examples/sec: 252.047
INFO:tensorflow:global_step/sec: 7.81502
INFO:tensorflow:examples/sec: 250.081
INFO:tensorflow:global_step/sec: 7.17396
INFO:tensorflow:examples/sec: 229.567
INFO:tensorflow:global_step/sec: 7.19653
INFO:tensorflow:examples/sec: 230.289
INFO:tensorflow:global_step/sec: 6.58105
INFO:tensorflow:examples/sec: 210.593
INFO:tensorflow:global_step/sec: 7.35528
INFO:tensorflow:examples/sec: 235.369
INFO:tensorflow:global_step/sec: 8.00255
INFO:tensorflow:examples/sec: 256.082
INFO:tensorflow:global_step/sec: 7.57815
INFO:tensorflow:examples/sec: 242.501
INFO:tensorflow:global_step/sec: 7.5782
INFO:tensorflow:examples/sec: 242.502
INFO:tensorflow:global_step/sec: 8.19933
INFO:tensorflow:examples/sec: 262.379
INFO:tensorflow:global_step/sec: 8.38088
INFO:tensorflow:examples/sec: 268.188
INFO:tensorflow

...

INFO:tensorflow:examples/sec: 225.423
INFO:tensorflow:global_step/sec: 7.35531
INFO:tensorflow:examples/sec: 235.37
INFO:tensorflow:global_step/sec: 7.19651
INFO:tensorflow:examples/sec: 230.288
INFO:tensorflow:global_step/sec: 7.19657
INFO:tensorflow:examples/sec: 230.29
INFO:tensorflow:global_step/sec: 7.19651
INFO:tensorflow:examples/sec: 230.288
INFO:tensorflow:global_step/sec: 7.14409
INFO:tensorflow:examples/sec: 228.611
INFO:tensorflow:global_step/sec: 7.52118
INFO:tensorflow:examples/sec: 240.678
INFO:tensorflow:global_step/sec: 7.40982
INFO:tensorflow:examples/sec: 237.114
INFO:tensorflow:global_step/sec: 7.30157
INFO:tensorflow:examples/sec: 233.65
INFO:tensorflow:global_step/sec: 7.09443
INFO:tensorflow:examples/sec: 227.022
INFO:tensorflow:global_step/sec: 7.04453
INFO:tensorflow:examples/sec: 225.425
INFO:tensorflow:global_step/sec: 7.09448
INFO:tensorflow:examples/sec: 227.023
INFO:tensorflow:global_step/sec: 6.89875
INFO:tensorflow:examples/sec: 220.76
INFO:tensorflow:gl

INFO:tensorflow:examples/sec: 217.757
INFO:tensorflow:global_step/sec: 7.57814
INFO:tensorflow:examples/sec: 242.501
INFO:tensorflow:global_step/sec: 6.75895
INFO:tensorflow:examples/sec: 216.286
INFO:tensorflow:global_step/sec: 7.69474
INFO:tensorflow:examples/sec: 246.232
INFO:tensorflow:global_step/sec: 7.35528
INFO:tensorflow:examples/sec: 235.369
INFO:tensorflow:global_step/sec: 7.63605
INFO:tensorflow:examples/sec: 244.353
INFO:tensorflow:global_step/sec: 7.636
INFO:tensorflow:examples/sec: 244.352
INFO:tensorflow:global_step/sec: 7.57811
INFO:tensorflow:examples/sec: 242.5
INFO:tensorflow:global_step/sec: 7.52126
INFO:tensorflow:examples/sec: 240.68
INFO:tensorflow:global_step/sec: 7.57818
INFO:tensorflow:examples/sec: 242.502
INFO:tensorflow:global_step/sec: 7.69477
INFO:tensorflow:examples/sec: 246.233
INFO:tensorflow:global_step/sec: 7.69473
INFO:tensorflow:examples/sec: 246.231
INFO:tensorflow:global_step/sec: 7.46505
INFO:tensorflow:examples/sec: 238.882
INFO:tensorflow:glo

INFO:tensorflow:examples/sec: 228.645
INFO:tensorflow:global_step/sec: 7.30159
INFO:tensorflow:examples/sec: 233.651
INFO:tensorflow:global_step/sec: 7.69476
INFO:tensorflow:examples/sec: 246.232
INFO:tensorflow:global_step/sec: 6.97017
INFO:tensorflow:examples/sec: 223.045
INFO:tensorflow:global_step/sec: 7.40975
INFO:tensorflow:examples/sec: 237.112
INFO:tensorflow:global_step/sec: 6.6687
INFO:tensorflow:examples/sec: 213.398
INFO:tensorflow:global_step/sec: 6.58115
INFO:tensorflow:examples/sec: 210.597
INFO:tensorflow:global_step/sec: 7.30159
INFO:tensorflow:examples/sec: 233.651
INFO:tensorflow:global_step/sec: 7.75438
INFO:tensorflow:examples/sec: 248.14
INFO:tensorflow:global_step/sec: 7.69474
INFO:tensorflow:examples/sec: 246.232
INFO:tensorflow:global_step/sec: 7.87657
INFO:tensorflow:examples/sec: 252.05
INFO:tensorflow:global_step/sec: 7.40976
INFO:tensorflow:examples/sec: 237.112
INFO:tensorflow:global_step/sec: 6.74121
INFO:tensorflow:examples/sec: 215.719
INFO:tensorflow:g

INFO:tensorflow:global_step/sec: 7.19657
INFO:tensorflow:examples/sec: 230.29
INFO:tensorflow:global_step/sec: 7.5212
INFO:tensorflow:examples/sec: 240.678
INFO:tensorflow:global_step/sec: 7.19656
INFO:tensorflow:examples/sec: 230.29
INFO:tensorflow:global_step/sec: 7.44156
INFO:tensorflow:examples/sec: 238.13
INFO:tensorflow:global_step/sec: 7.4651
INFO:tensorflow:examples/sec: 238.883
INFO:tensorflow:global_step/sec: 7.17816
INFO:tensorflow:examples/sec: 229.701
INFO:tensorflow:global_step/sec: 7.0445
INFO:tensorflow:examples/sec: 225.424
INFO:tensorflow:global_step/sec: 7.46507
INFO:tensorflow:examples/sec: 238.882
INFO:tensorflow:global_step/sec: 7.3016
INFO:tensorflow:examples/sec: 233.651
INFO:tensorflow:global_step/sec: 7.24869
INFO:tensorflow:examples/sec: 231.958
INFO:tensorflow:global_step/sec: 7.24864
INFO:tensorflow:examples/sec: 231.957
INFO:tensorflow:global_step/sec: 7.55338
INFO:tensorflow:examples/sec: 241.708
INFO:tensorflow:global_step/sec: 6.88184
INFO:tensorflow:ex

INFO:tensorflow:examples/sec: 231.959
INFO:tensorflow:global_step/sec: 7.57817
INFO:tensorflow:examples/sec: 242.501
INFO:tensorflow:global_step/sec: 7.46502
INFO:tensorflow:examples/sec: 238.881
INFO:tensorflow:global_step/sec: 7.5782
INFO:tensorflow:examples/sec: 242.502
INFO:tensorflow:global_step/sec: 7.01896
INFO:tensorflow:examples/sec: 224.607
INFO:tensorflow:global_step/sec: 7.63602
INFO:tensorflow:examples/sec: 244.353
INFO:tensorflow:global_step/sec: 7.52116
INFO:tensorflow:examples/sec: 240.677
INFO:tensorflow:global_step/sec: 7.6948
INFO:tensorflow:examples/sec: 246.234
INFO:tensorflow:global_step/sec: 6.94665
INFO:tensorflow:examples/sec: 222.293
INFO:tensorflow:global_step/sec: 7.46505
INFO:tensorflow:examples/sec: 238.882
INFO:tensorflow:global_step/sec: 7.19653
INFO:tensorflow:examples/sec: 230.289
INFO:tensorflow:global_step/sec: 7.30153
INFO:tensorflow:examples/sec: 233.649
INFO:tensorflow:global_step/sec: 7.35534
INFO:tensorflow:examples/sec: 235.371
INFO:tensorflow:

In [12]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    
    #batch_size = params["batch_size"]
    batch_size = 64 # <----- hardcoded batch_size added here 
    
    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn

In [13]:
start = time.time()
print("--------------------------------------------------------")
print("Starting inference ...")
print("--------------------------------------------------------")

predict_examples = processor.get_test_examples(data_dir)
num_actual_predict_examples = len(predict_examples)

predict_file = os.path.join(output_dir, "predict.tf_record")

file_based_convert_examples_to_features(predict_examples, label_list,
                                        max_seq_length, tokenizer,
                                        predict_file)

tf.logging.info("***** Running prediction*****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(predict_examples), num_actual_predict_examples,
                len(predict_examples) - num_actual_predict_examples)
tf.logging.info("  Batch size = %d", predict_batch_size)

predict_drop_remainder = True if use_tpu else False
predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=max_seq_length,
    is_training=False,
    drop_remainder=predict_drop_remainder)

result = estimator.predict(input_fn=predict_input_fn)

output_predict_file = os.path.join(output_dir, "test_results.tsv")

with tf.gfile.GFile(output_predict_file, "w") as writer:
    num_written_lines = 0
    tf.logging.info("***** Predict results *****")
    for (i, prediction) in enumerate(result):
        probabilities = prediction["probabilities"]
        if i >= num_actual_predict_examples:
            break
        output_line = "\t".join(
            str(class_probability)
            for class_probability in probabilities) + "\n"
        writer.write(output_line)
        num_written_lines += 1
        
end = time.time()
print("--------------------------------------------------------")
print("Inference complete in ", end - start, " seconds")
print("--------------------------------------------------------")

--------------------------------------------------------
Starting inference ...
--------------------------------------------------------
INFO:tensorflow:Writing example 0 of 47124
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-1
INFO:tensorflow:tokens: [CLS] 7 [SEP]
INFO:tensorflow:input_ids: 101 1021 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-2
INFO:tensorflow:tokens: [CLS] 4 [SEP]
INFO:tensorflow:input_ids: 101 1018 102 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:  name = bert/pooler/dense/kernel:0, shape = (128, 128), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/pooler/dense/bias:0, shape = (128,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = output_weights:0, shape = (12, 128)
INFO:tensorflow:  name = output_bias:0, shape = (12,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./output\model.ckpt-5890
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:prediction_loop marked as finished
INFO:tensorflow:prediction_loop marked as finished
--------------------------------------------------------
Inference complete in  129.1498281955719  seconds
--------------------------------------------------------
